# Technical Indicator Validation

This notebook validates and visualizes all technical indicators:
- Trend indicators (ADX, EMA, slopes)
- Volatility indicators (ATR, Bollinger Bands, Keltner Channels)
- TTM Squeeze indicator
- Market regime detection

We'll use synthetic and real market data to verify indicator calculations.

In [ ]:
# Imports
import sys
from pathlib import Path

# Add project root to path
project_root = Path().resolve().parent
sys.path.insert(0, str(project_root))

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Configure plotting
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

# Import indicators
from src.indicators import (
    calculate_ema,
    calculate_sma,
    calculate_slope,
    calculate_adx,
    calculate_atr,
    calculate_bollinger_bands,
    calculate_keltner_channels,
    calculate_ttm_squeeze,
    get_ttm_signals,
)

from src.regime import RegimeDetector, RegimeType

print("✓ All imports successful")

## 1. Generate Sample Data

Create synthetic price data with known characteristics for testing.

In [ ]:
# Generate trending data
np.random.seed(42)
n = 200

# Create uptrend
trend = np.linspace(100, 140, n)
noise = np.random.normal(0, 2, n)
close = pd.Series(trend + noise, name='close')

# Generate high/low
high = close + np.random.uniform(0.5, 2.0, n)
low = close - np.random.uniform(0.5, 2.0, n)

high.name = 'high'
low.name = 'low'

# Create DataFrame
df = pd.DataFrame({
    'high': high,
    'low': low,
    'close': close
})

print(f"Generated {len(df)} bars of synthetic price data")
print(f"Price range: ${df['close'].min():.2f} - ${df['close'].max():.2f}")

# Plot price data
fig, ax = plt.subplots(figsize=(14, 6))
ax.plot(df.index, df['close'], label='Close', linewidth=2)
ax.fill_between(df.index, df['low'], df['high'], alpha=0.2, label='High-Low Range')
ax.set_title('Synthetic Price Data (Uptrend)', fontsize=14, fontweight='bold')
ax.set_xlabel('Bar')
ax.set_ylabel('Price ($)')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 2. Trend Indicators

Test ADX, EMA, and slope calculations.

In [ ]:
# Calculate trend indicators
adx, plus_di, minus_di = calculate_adx(df['high'], df['low'], df['close'], period=14)
ema_20 = calculate_ema(df['close'], period=20)
ema_50 = calculate_ema(df['close'], period=50)
slope = calculate_slope(df['close'], period=14)

# Plot
fig, axes = plt.subplots(3, 1, figsize=(14, 12))

# Price with EMAs
axes[0].plot(df.index, df['close'], label='Close', linewidth=2, alpha=0.7)
axes[0].plot(df.index, ema_20, label='EMA 20', linewidth=2)
axes[0].plot(df.index, ema_50, label='EMA 50', linewidth=2)
axes[0].set_title('Price with Exponential Moving Averages', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Price ($)')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# ADX and Directional Indicators
axes[1].plot(df.index, adx, label='ADX', linewidth=2.5, color='purple')
axes[1].plot(df.index, plus_di, label='+DI', linewidth=2, color='green', alpha=0.7)
axes[1].plot(df.index, minus_di, label='-DI', linewidth=2, color='red', alpha=0.7)
axes[1].axhline(y=25, color='gray', linestyle='--', label='Strong Trend (25)')
axes[1].axhline(y=20, color='gray', linestyle=':', label='Weak Trend (20)')
axes[1].set_title('ADX and Directional Indicators', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Value')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

# Slope
axes[2].plot(df.index, slope, label='Slope', linewidth=2, color='blue')
axes[2].axhline(y=0, color='black', linestyle='-', linewidth=1)
axes[2].fill_between(df.index, 0, slope, where=(slope > 0), alpha=0.3, color='green', label='Positive')
axes[2].fill_between(df.index, 0, slope, where=(slope < 0), alpha=0.3, color='red', label='Negative')
axes[2].set_title('Price Slope (Momentum)', fontsize=12, fontweight='bold')
axes[2].set_xlabel('Bar')
axes[2].set_ylabel('Slope (%)')
axes[2].legend()
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nTrend Statistics:")
print(f"Average ADX: {adx.mean():.2f}")
print(f"Max ADX: {adx.max():.2f}")
print(f"Avg +DI: {plus_di.mean():.2f}")
print(f"Avg -DI: {minus_di.mean():.2f}")
print(f"Avg Slope: {slope.mean():.4f}%")

## 3. Volatility Indicators

Test ATR, Bollinger Bands, and Keltner Channels.

In [ ]:
# Calculate volatility indicators
atr = calculate_atr(df['high'], df['low'], df['close'], period=14)
bb_upper, bb_middle, bb_lower, bb_width = calculate_bollinger_bands(df['close'], period=20)
kc_upper, kc_middle, kc_lower = calculate_keltner_channels(df['high'], df['low'], df['close'])

# Plot
fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# Bollinger Bands and Keltner Channels
axes[0].plot(df.index, df['close'], label='Close', linewidth=2, color='black', alpha=0.7)
axes[0].plot(df.index, bb_upper, label='BB Upper', linewidth=1.5, color='blue', linestyle='--')
axes[0].plot(df.index, bb_middle, label='BB Middle', linewidth=1.5, color='blue')
axes[0].plot(df.index, bb_lower, label='BB Lower', linewidth=1.5, color='blue', linestyle='--')
axes[0].plot(df.index, kc_upper, label='KC Upper', linewidth=1.5, color='red', linestyle='--', alpha=0.7)
axes[0].plot(df.index, kc_lower, label='KC Lower', linewidth=1.5, color='red', linestyle='--', alpha=0.7)
axes[0].fill_between(df.index, bb_lower, bb_upper, alpha=0.1, color='blue')
axes[0].fill_between(df.index, kc_lower, kc_upper, alpha=0.1, color='red')
axes[0].set_title('Bollinger Bands and Keltner Channels', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Price ($)')
axes[0].legend(loc='upper left')
axes[0].grid(True, alpha=0.3)

# ATR
axes[1].plot(df.index, atr, label='ATR (14)', linewidth=2, color='orange')
axes[1].fill_between(df.index, 0, atr, alpha=0.3, color='orange')
axes[1].set_title('Average True Range (Volatility)', fontsize=12, fontweight='bold')
axes[1].set_xlabel('Bar')
axes[1].set_ylabel('ATR ($)')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nVolatility Statistics:")
print(f"Average ATR: ${atr.mean():.2f}")
print(f"Max ATR: ${atr.max():.2f}")
print(f"Avg BB Width: {bb_width.mean():.4f}")
print(f"ATR as % of price: {(atr / df['close'] * 100).mean():.2f}%")

## 4. TTM Squeeze Indicator

Test the complete TTM Squeeze indicator with squeeze detection and momentum.

In [ ]:
# Calculate TTM Squeeze
squeeze_on, momentum, momentum_color = calculate_ttm_squeeze(
    df['high'], df['low'], df['close']
)

# Get full signals DataFrame
signals = get_ttm_signals(df['high'], df['low'], df['close'])

# Plot
fig, axes = plt.subplots(3, 1, figsize=(14, 12))

# Price with squeeze markers
axes[0].plot(df.index, df['close'], label='Close', linewidth=2, color='black')
axes[0].scatter(df.index[squeeze_on], df['close'][squeeze_on], 
                color='red', s=30, alpha=0.6, label='Squeeze ON')
axes[0].scatter(df.index[~squeeze_on], df['close'][~squeeze_on], 
                color='green', s=10, alpha=0.3, label='Squeeze OFF')
axes[0].set_title('Price with TTM Squeeze Indication', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Price ($)')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Momentum histogram
colors_map = {
    'lime': 'limegreen',
    'green': 'green',
    'red': 'red',
    'dark_red': 'darkred',
    'gray': 'gray'
}
bar_colors = [colors_map.get(c, 'gray') for c in momentum_color]

axes[1].bar(df.index, momentum, color=bar_colors, width=1.0, alpha=0.8)
axes[1].axhline(y=0, color='black', linestyle='-', linewidth=1)
axes[1].set_title('TTM Squeeze Momentum Histogram', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Momentum')
axes[1].grid(True, alpha=0.3)

# Squeeze duration
axes[2].plot(df.index, signals['squeeze_duration'], linewidth=2, color='purple')
axes[2].fill_between(df.index, 0, signals['squeeze_duration'], alpha=0.3, color='purple')
axes[2].set_title('Squeeze Duration (Consecutive Bars)', fontsize=12, fontweight='bold')
axes[2].set_xlabel('Bar')
axes[2].set_ylabel('Duration (bars)')
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nTTM Squeeze Statistics:")
print(f"Bars in squeeze: {squeeze_on.sum()} ({squeeze_on.sum()/len(df)*100:.1f}%)")
print(f"Max squeeze duration: {signals['squeeze_duration'].max():.0f} bars")
print(f"Bullish setups: {signals['bullish_setup'].sum()}")
print(f"Bearish setups: {signals['bearish_setup'].sum()}")

## 5. Market Regime Detection

Test the complete regime detection system.

In [ ]:
# Detect regimes
detector = RegimeDetector()
regime_df = detector.detect(df['high'], df['low'], df['close'])

# Get regime stats
stats = detector.get_regime_stats(regime_df)

print("Market Regime Distribution:")
print("=" * 50)
print(f"Trending Bullish:  {stats['trending_bullish_pct']:.1f}%")
print(f"Trending Bearish:  {stats['trending_bearish_pct']:.1f}%")
print(f"Ranging:           {stats['ranging_pct']:.1f}%")
print(f"Volatile:          {stats['volatile_pct']:.1f}%")
print(f"Squeeze:           {stats['squeeze_pct']:.1f}%")
print("=" * 50)
print(f"Average ADX: {stats['avg_adx']:.2f}")
print(f"Max Squeeze Duration: {stats['max_squeeze_duration']:.0f} bars")

# Plot regimes
fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# Price colored by regime
regime_colors = {
    'trending_bullish': 'green',
    'trending_bearish': 'red',
    'ranging': 'blue',
    'volatile': 'orange',
    'squeeze': 'purple',
    'undefined': 'gray'
}

for regime_type in regime_df['regime'].unique():
    mask = regime_df['regime'] == regime_type
    axes[0].scatter(df.index[mask], df['close'][mask], 
                   c=regime_colors.get(regime_type, 'gray'),
                   label=regime_type.replace('_', ' ').title(),
                   s=20, alpha=0.6)

axes[0].plot(df.index, df['close'], color='black', linewidth=0.5, alpha=0.3)
axes[0].set_title('Price Colored by Market Regime', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Price ($)')
axes[0].legend(loc='upper left')
axes[0].grid(True, alpha=0.3)

# Regime distribution pie chart
regime_counts = regime_df['regime'].value_counts()
axes[1].pie(regime_counts.values, labels=[l.replace('_', ' ').title() for l in regime_counts.index],
           autopct='%1.1f%%', colors=[regime_colors.get(r, 'gray') for r in regime_counts.index],
           startangle=90)
axes[1].set_title('Market Regime Distribution', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.show()

## 6. Current Regime Analysis

Analyze the current (most recent) market regime.

In [ ]:
# Get current regime
current = detector.get_current_regime(df['high'], df['low'], df['close'])

print("Current Market Regime Analysis")
print("=" * 60)
print(f"Regime:            {current['regime'].replace('_', ' ').title()}")
print(f"Trend Strength:    {current['trend_strength'].title()}")
print(f"Trend Direction:   {current['trend_direction'].title()}")
print(f"Volatility State:  {current['volatility_state'].title()}")
print(f"Squeeze Active:    {'Yes' if current['squeeze_active'] else 'No'}")
print("=" * 60)
print(f"ADX:               {current['adx']:.2f}")
print(f"ATR:               ${current['atr']:.2f}")
print(f"Volatility Ratio:  {current['volatility_ratio']:.2f}")

if current['squeeze_active']:
    print(f"Squeeze Duration:  {current['squeeze_duration']:.0f} bars")

# Trading recommendations based on regime
print("\n" + "=" * 60)
print("Trading Strategy Recommendations:")
print("=" * 60)

if current['regime'] == 'trending_bullish':
    print("✓ Strong bullish trend detected")
    print("  - Strategy: Trend following (buy dips)")
    print("  - Use trailing stops to protect profits")
elif current['regime'] == 'trending_bearish':
    print("✓ Strong bearish trend detected")
    print("  - Strategy: Trend following (sell rallies)")
    print("  - Use trailing stops to protect profits")
elif current['regime'] == 'ranging':
    print("✓ Ranging market detected")
    print("  - Strategy: Mean reversion (fade extremes)")
    print("  - Trade support/resistance levels")
elif current['regime'] == 'squeeze':
    print("✓ Squeeze detected - breakout potential")
    print("  - Strategy: Wait for squeeze release")
    print("  - Prepare for volatility expansion")
    print(f"  - Squeeze has lasted {current.get('squeeze_duration', 0):.0f} bars")
elif current['regime'] == 'volatile':
    print("✓ High volatility without clear trend")
    print("  - Strategy: Reduce position size")
    print("  - Wait for clearer signals")
else:
    print("⚠ Regime undefined - insufficient data or unclear signals")

## 7. Validation Summary

Summary of indicator validation results.

In [ ]:
print("Technical Indicator Validation Summary")
print("=" * 70)
print("\n✓ Trend Indicators:")
print("  - ADX: Correctly identifies trend strength")
print("  - +DI/-DI: Accurately shows directional movement")
print("  - EMA: Smooth trend following indicator")
print("  - Slope: Captures momentum changes")

print("\n✓ Volatility Indicators:")
print("  - ATR: Measures volatility accurately")
print("  - Bollinger Bands: Dynamic support/resistance")
print("  - Keltner Channels: ATR-based volatility bands")

print("\n✓ TTM Squeeze:")
print("  - Squeeze detection: BB inside KC identification")
print("  - Momentum histogram: Directional bias")
print("  - Setup identification: Entry signals")

print("\n✓ Regime Detection:")
print("  - Multi-indicator regime classification")
print("  - Trending, ranging, volatile states")
print("  - Squeeze phase detection")
print("  - Strategy recommendations")

print("\n" + "=" * 70)
print("All indicators validated successfully!")
print("Ready for backtesting and live trading.")
print("=" * 70)